In [1]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

C:\Users\shind\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key="pcsk_uQZa5_6sh7ivL9LdoTKHrz5BKiyan8VcR9QUfeB1R8eBnVj2nPnnpGJwJToBMW3BT2Dnb")

In [3]:
data = [
    {"id": "vec1", "text": "Table: author; Columns: aid, homepage, name, oid;"},
    {"id": "vec2", "text": "Table: conference; Columns: cid, homepage, name;"},
    {"id": "vec3", "text": "Table: domain; Columns: did, name;"},
    {"id": "vec4", "text": "Table: domain_author; Columns: aid, did;"},
    {"id": "vec5", "text": "Table: domain_conference; Columns: cid, did;"},
    {"id": "vec6", "text": "Table: journal; Columns: homepage, jid, name;"},
    {"id": "vec7", "text": "Table: domain_journal; Columns: did, jid;"},
    {"id": "vec8", "text": "Table: keyword; Columns: keyword, kid;"},
    {"id": "vec9", "text": "Table: domain_keyword; Columns: did, kid;"},
    {"id": "vec10", "text": "Table: publication; Columns: abstract, cid, citation_num, jid, pid, reference_num, title, year;"},
    {"id": "vec11", "text": "Table: domain_publication; Columns: did, pid;"},
    {"id": "vec12", "text": "Table: organization; Columns: continent, homepage, name, oid;"},
    {"id": "vec13", "text": "Table: publication_keyword; Columns: pid, kid;"},
    {"id": "vec14", "text": "Table: writes; Columns: aid, pid;"},
    {"id": "vec15", "text": "Table: cite; Columns: cited, citing;"}
]



In [9]:
data += [
    {"id": "vec16", "text": "Table: Activity; Columns: actid, activity_name;"},
    {"id": "vec17", "text": "Table: Participates_in; Columns: stuid, actid;"},
    {"id": "vec18", "text": "Table: Faculty_Participates_in; Columns: FacID, actid;"},
    {"id": "vec19", "text": "Table: Student; Columns: StuID, LName, FName, Age, Sex, Major, Advisor, city_code;"},
    {"id": "vec20", "text": "Table: Faculty; Columns: FacID, LName, FName, Rank, Sex, Phone, Room, Building;"}
]

In [10]:
data += [
    {"id": "vec21", "text": "Table: Student; Columns: StuID, LName, Fname, Age, Sex, Major, Advisor, city_code;"},
    {"id": "vec22", "text": "Table: Direct_distance; Columns: city1_code, city2_code, distance;"},
    {"id": "vec23", "text": "Table: City; Columns: city_code, city_name, state, country, latitude, longitude;"}
]


In [11]:
data += [
    {"id": "vec24", "text": "Table: City; Columns: city_code, city_name, state, country, latitude, longitude;"},
    {"id": "vec25", "text": "Table: University; Columns: uni_code, uni_name, city_code, state, country;"},
    {"id": "vec26", "text": "Table: Course; Columns: course_code, course_name, credits, department, uni_code;"},
    {"id": "vec27", "text": "Table: Enrollment; Columns: enroll_id, StuID, course_code, semester, year, grade;"},
    {"id": "vec28", "text": "Table: Department; Columns: dept_code, dept_name, uni_code, chairperson;"}
]


In [16]:
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

In [17]:
index = pc.Index("schema")

records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=records,
    namespace="example-namespace"
)


upserted_count: 28

In [15]:
len(records)

15

In [13]:
query = "What is domain of journal with name IEEE?"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec7',
              'metadata': {'text': 'Table: domain_journal; Columns: did, jid;'},
              'score': 0.7661627,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec6',
              'metadata': {'text': 'Table: journal; Columns: homepage, jid, '
                                   'name;'},
              'score': 0.76058525,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec10',
              'metadata': {'text': 'Table: publication; Columns: abstract, '
                                   'cid, citation_num, jid, pid, '
                                   'reference_num, title, year;'},
              'score': 0.7589219,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'example-namespace',
 'usage': {'read_units': 6}}


In [19]:
results['matches'][0]['metadata']['text']

'Table: domain_journal; Columns: did, jid;'